# ❄️ Snowflake ML Function: Anomaly Detection Example ❄️

In [ ]:
# Streamlit
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

# Snowpark
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
use schema HOL.PUBLIC;

In [ ]:
-- Create a view containing our training data
CREATE OR REPLACE VIEW anomaly_training_set AS (
    SELECT *
    FROM sales_forecast_input
    WHERE timestamp < (SELECT MAX(timestamp) FROM sales_forecast_input) - interval '1 Month'
);

-- Create a view containing the data we want to make inferences on
CREATE OR REPLACE VIEW anomaly_analysis_set AS (
    SELECT *
    FROM sales_forecast_input
    WHERE timestamp > (SELECT MAX(timestamp) FROM anomaly_training_set)
);


In [ ]:
SELECT * FROM anomaly_training_set limit 10;

In [ ]:
SELECT MIN(TIMESTAMP) as Min, MAX(TIMESTAMP) as Max, Count(TIMESTAMP) as Count FROM ANOMALY_TRAINING_SET;

In [ ]:
df_test = session.table('ANOMALY_TRAINING_SET')
st.line_chart(df_test, x= 'TIMESTAMP', y= 'TOTAL_SOLD')

In [ ]:
CREATE OR REPLACE VIEW ANOMALY_TRAINING as
select to_timestamp_ntz(TIMESTAMP) as TIMESTAMP,
    TOTAL_SOLD,
    MENU_ITEM_NAME
FROM ANOMALY_TRAINING_SET
WHERE timestamp > (SELECT MAX(TO_DATE(timestamp)) - interval '2 Months' FROM ANOMALY_TRAINING_SET);

In [ ]:
SELECT MIN(TO_DATE(TIMESTAMP)) as Min_Date, MAX(TO_DATE(TIMESTAMP)) as Max_Date, COUNT(TIMESTAMP) as Count from ANOMALY_TRAINING;

In [ ]:
SELECT MIN(TO_DATE(TIMESTAMP)) as Min_Date, MAX(TO_DATE(TIMESTAMP)) as Max_Date FROM ANOMALY_ANALYSIS_SET;

In [ ]:
-- Create the model: UNSUPERVISED method, however can pass labels as well; this could take ~15-25 secs; please be patient 
CREATE OR REPLACE snowflake.ml.anomaly_detection anomaly_model(
    INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'ANOMALY_TRAINING'),
    SERIES_COLNAME => 'MENU_ITEM_NAME',
    TIMESTAMP_COLNAME => 'TIMESTAMP',
    TARGET_COLNAME => 'TOTAL_SOLD',
    LABEL_COLNAME => ''
);

In [ ]:
CREATE OR REPLACE TABLE anomalies AS 
      SELECT * FROM TABLE( anomaly_model!DETECT_ANOMALIES(
                                        INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'ANOMALY_ANALYSIS_SET'),
                                        SERIES_COLNAME => 'MENU_ITEM_NAME',
                                        TIMESTAMP_COLNAME => 'TIMESTAMP',
                                        TARGET_COLNAME => 'TOTAL_SOLD',
                                        CONFIG_OBJECT => {'prediction_interval': 0.95}
                                    )
);

In [ ]:
-- Call the model and store the results into table; this could take ~10-20 secs; please be patient
CALL anomaly_model!DETECT_ANOMALIES(
    INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'ANOMALY_ANALYSIS_SET'),
    SERIES_COLNAME => 'MENU_ITEM_NAME',
    TIMESTAMP_COLNAME => 'TIMESTAMP',
    TARGET_COLNAME => 'TOTAL_SOLD',
    CONFIG_OBJECT => {'prediction_interval': 0.95}
);

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()
code_to_run = """
BEGIN
    -- This is the step that creates your predictions.
    CALL anomaly_model!DETECT_ANOMALIES(
        INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'ANOMALY_ANALYSIS_SET'),
        SERIES_COLNAME => 'MENU_ITEM_NAME',
        TIMESTAMP_COLNAME => 'TIMESTAMP',
        TARGET_COLNAME => 'TOTAL_SOLD',
        
        -- Here we set your prediction interval.
        CONFIG_OBJECT => {'prediction_interval': 0.95}
    );
    -- These steps store your predictions to a table.
    LET x := SQLID;
    CREATE OR REPLACE TABLE anomalies AS SELECT * FROM TABLE(RESULT_SCAN(:x));
END;
"""
data = session.sql(code_to_run).collect(block=True);

In [ ]:
select * from anomalies limit 10;

In [ ]:
-- Query to identify trends
SELECT series, is_anomaly, count(is_anomaly) AS num_records
FROM anomalies
WHERE is_anomaly =1
GROUP BY ALL
ORDER BY num_records DESC
LIMIT 5;

In [ ]:
CREATE OR REPLACE VIEW sales_anomalies AS (
    SELECT 
        A.TIMESTAMP AS sales_date, 
        A.MENU_ITEM_NAME, 
        A.TOTAL_SOLD, 
        COALESCE(B.IS_ANOMALY, FALSE) AS IS_ANOMALY ,
        IFF(B.IS_ANOMALY, 20, 0) as ANOMALY_VAL
    FROM 
        ANOMALY_ANALYSIS_SET A
    LEFT JOIN 
        anomalies B 
    ON 
        A.MENU_ITEM_NAME = REGEXP_REPLACE(B.SERIES, '["]') AND A.TIMESTAMP = B.TS
    ORDER BY 
        sales_date, 
        MENU_ITEM_NAME
);

In [ ]:
select * from sales_anomalies;

In [ ]:
SELECT COUNT(sales_date) as Count, IS_ANOMALY 
FROM sales_anomalies 
GROUP BY IS_ANOMALY;

In [ ]:
SELECT MENU_ITEM_NAME, count(1) as Anamoly_Count
FROM sales_anomalies
WHERE IS_ANOMALY  =1 group by all;

In [ ]:
df = session.table('sales_anomalies').to_pandas()

# Convert 'SALES_DATE' to datetime if it's not already in datetime format
df['SALES_DATE'] = pd.to_datetime(df['SALES_DATE'])

# Calculate cumulative sum of TOTAL_SOLD for each MENU_ITEM_NAME
df['cumulative_total_sold'] = df.groupby(['SALES_DATE', 'MENU_ITEM_NAME'])['TOTAL_SOLD'].cumsum()

# Get unique MENU_ITEM_NAME values
menu_items = df['MENU_ITEM_NAME'].unique()

# Plotting
plt.figure(figsize=(12, 8))

for menu_item in menu_items:
    plt.plot(df[df['MENU_ITEM_NAME'] == menu_item]['SALES_DATE'], 
             df[df['MENU_ITEM_NAME'] == menu_item]['cumulative_total_sold'], 
             label=menu_item)

plt.xlabel('Date')
plt.ylabel('Cumulative Total Sold')
plt.title('Cumulative Total Sold Over Time for Each Menu Item')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=True, ncol=5)
plt.xticks(rotation=45)
plt.show()

In [ ]:
df = session.table('sales_anomalies').to_pandas()

# Filter menu items with > 0 cumulative anomalies
menu_items = df[df['IS_ANOMALY'] > 0]['MENU_ITEM_NAME'].unique()

# Plotting
plt.figure(figsize=(12, 8))

for menu_item in menu_items:
    df_menu_item = df[df['MENU_ITEM_NAME'] == menu_item]
    plt.bar(df_menu_item['SALES_DATE'], 
            df_menu_item['IS_ANOMALY'], 
            label=menu_item)

plt.xlabel('Date')
plt.ylabel('Anomalies')
plt.title('Anomalies by Menu Item')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=True, ncol=5)
plt.show()

In [ ]:
import plotly.express as px
from snowflake.snowpark.functions import col
#session = get_active_session()
df = session.table('sales_anomalies')

df = df.filter(  col("MENU_ITEM_NAME") == "Italian") 

st.plotly_chart(px.line(df, x="SALES_DATE" ,y=["TOTAL_SOLD", "ANOMALY_VAL"]), use_container_width=True)


          





In [ ]:
SELECT * FROM sales_anomalies where MENU_ITEM_NAME= 'Italian'